# Conversational Context with LangGraph

Building multi-turn conversations where the agent maintains context across user interactions.

## Learning Objectives

By the end of this notebook, you will:

1. **Manage conversational state** - Understand how MessagesState accumulates context across multiple user turns
2. **Handle pronoun references** - Enable the agent to understand references like "that RAV4" or "those two" by maintaining conversation history
3. **Build multi-turn workflows** - Create natural conversation flows for complex scenarios like car buying journeys
4. **Track conversation growth** - Monitor how state expands with each interaction and understand message accumulation patterns

## 1. Environment Setup

In [ ]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.documents import Document
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

from dotenv import load_dotenv
from typing import Literal, List

load_dotenv("../../.env")
print("✅ Environment loaded")

In [ ]:
# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

print("✅ LLM initialized")

## 2. Connect to Vector Store

In [ ]:
# ChromaDB Configuration
PERSIST_DIR = "../../chroma_db"
COLLECTION_NAME = "toyota_specs"
EMBED_MODEL_ID = "gemini-embedding-001"

# Initialize embeddings
embeddings_model = GoogleGenerativeAIEmbeddings(
    model=EMBED_MODEL_ID,
    output_dimensionality=768
)

# Connect to vectorstore
vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=embeddings_model,
    persist_directory=PERSIST_DIR
)

print(f"✅ Connected to vectorstore: {COLLECTION_NAME}")
print(f"   Total documents: {vectorstore._collection.count()}")

## 3. Define Tools

In [ ]:
# Vector similarity search helper
def vector_similarity_search(
    query: str, 
    vectorstore, 
    k: int = 5
) -> List[str]:
    """Perform vector similarity search."""
    docs = vectorstore.similarity_search(query, k=k)
    return [doc.page_content for doc in docs]

In [ ]:
# Tool 1: Vehicle Search
@tool
def search_vehicles(query: str, max_results: int = 5) -> str:
    """
    Search Toyota vehicle database using semantic similarity.
    
    Use this tool when users need information about Toyota vehicles,
    including specifications, pricing, fuel efficiency, or comparisons.
    
    Args:
        query: Natural language search query about Toyota vehicles
        max_results: Maximum number of results to return (default: 5)
    
    Returns:
        Formatted string with vehicle information
    """
    docs = vector_similarity_search(query, vectorstore, k=max_results)
    
    result = "Vehicle Search Results:\n"
    result += "=" * 60 + "\n"
    for i, doc in enumerate(docs, 1):
        result += f"\nResult {i}:\n{doc}\n"
    result += "=" * 60
    
    return result

print("✅ search_vehicles tool defined")

In [ ]:
# Tool 2: EMI Calculator
@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    
    Args:
        principal: The loan amount
        annual_interest_rate: Annual interest rate as percentage (e.g., 8.5)
        tenure_months: Loan tenure in months
        currency: Currency code (USD, EUR, GBP, INR, JPY)
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ emi_calculator tool defined")

## 4. Build LangGraph Workflow

In [ ]:
# Initialize LLM with tools
tools = [search_vehicles, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

def call_llm(state: MessagesState):
    """LLM node: Calls LLM with current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router: Check if agent wants to use tools."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("llm", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "llm")
workflow.add_conditional_edges("llm", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "llm")

app = workflow.compile()
print("✅ Graph compiled")

## 5. Understanding Conversational Context

### What is Conversational Context?

Conversational context is the ability to maintain information across multiple user turns in a conversation. This enables:

**1. Pronoun Resolution**
- User: "Tell me about the RAV4"
- Agent: [provides RAV4 info]
- User: "What's the monthly payment for **it**?" ← Agent knows "it" = RAV4

**2. Reference to Previous Results**
- User: "Compare Camry and RAV4 prices"
- Agent: [shows both prices]
- User: "Calculate EMI for **those two**" ← Agent knows which vehicles

**3. Natural Follow-up Questions**
- User: "Show me SUVs"
- Agent: [lists SUVs]
- User: "Which one has the best mileage?" ← Agent knows we're still talking about SUVs

### How MessagesState Enables This

The `MessagesState` in LangGraph accumulates ALL messages from the conversation:
- Every `HumanMessage` (user query)
- Every `AIMessage` (agent response or tool call)
- Every `ToolMessage` (tool result)

This growing list of messages is passed to the LLM on each turn, giving it full conversation history.

### Key Pattern for Multi-Turn Conversations

```python
# Initialize conversation state
conversation = {"messages": []}

# Turn 1
conversation["messages"].append(HumanMessage(content="First query"))
conversation = app.invoke(conversation)  # State now has 3-4 messages

# Turn 2 - State persists!
conversation["messages"].append(HumanMessage(content="Follow-up query"))
conversation = app.invoke(conversation)  # State now has 6-8 messages
```

The state **accumulates** - it doesn't reset between turns!

## 6. Multi-Turn Car Buying Conversation

Let's simulate a realistic car buying journey with multiple turns.

In [ ]:
# Initialize conversation
conversation = {"messages": []}

print("MULTI-TURN CONVERSATIONAL CONTEXT TEST")
print("=" * 80)
print("Scenario: Car buying journey with context maintained across turns\n")

### Turn 1: Initial Vehicle Search

In [ ]:
# Turn 1: Find vehicles
print("TURN 1")
print("─" * 80)

user_query_1 = "I'm looking for an affordable Toyota SUV with good fuel economy. What do you recommend?"
print(f"👤 User: {user_query_1}")

conversation["messages"].append(HumanMessage(content=user_query_1))
conversation = app.invoke(conversation)

print(f"\n🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")

### Turn 2: EMI Calculation for Mentioned Vehicle

Notice the pronoun reference: "**that RAV4**" - the agent must remember the RAV4 from Turn 1.

In [ ]:
# Turn 2: Calculate EMI for recommended vehicle
print("\n" + "─" * 80)
print("TURN 2")
print("─" * 80)

user_query_2 = "What would the monthly payment be for that RAV4 at 6.5% interest over 5 years?"
print(f"👤 User: {user_query_2}")
print(f"\n   💡 Note: 'that RAV4' references Turn 1's recommendation!\n")

conversation["messages"].append(HumanMessage(content=user_query_2))
conversation = app.invoke(conversation)

print(f"🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")

### Turn 3: Comparison Request

The user asks for a comparison, establishing new context for the next turn.

In [ ]:
# Turn 3: Compare vehicles
print("\n" + "─" * 80)
print("TURN 3")
print("─" * 80)

user_query_3 = "Can you compare the base prices of the RAV4 and Highlander for me?"
print(f"👤 User: {user_query_3}\n")

conversation["messages"].append(HumanMessage(content=user_query_3))
conversation = app.invoke(conversation)

print(f"🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")

### Turn 4: Reference to Previous Comparison

The phrase "**those two**" references the RAV4 and Highlander from Turn 3. The agent must maintain context to understand this reference.

In [ ]:
# Turn 4: Calculate for compared vehicles
print("\n" + "─" * 80)
print("TURN 4")
print("─" * 80)

user_query_4 = "What would be the monthly payment difference between those two at 6% for 60 months?"
print(f"👤 User: {user_query_4}")
print(f"\n   💡 Note: 'those two' references RAV4 & Highlander from Turn 3!\n")

conversation["messages"].append(HumanMessage(content=user_query_4))
conversation = app.invoke(conversation)

print(f"🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")

## 7. Analyze Complete Conversation State

Let's examine the full conversation state to see how context accumulates.

In [ ]:
# Examine conversation state
print("COMPLETE CONVERSATION STATE")
print("=" * 80)

turn_num = 0
for i, msg in enumerate(conversation["messages"], 1):
    if isinstance(msg, HumanMessage):
        turn_num += 1
        print(f"\n{'─' * 80}")
        print(f"TURN {turn_num}")
        print(f"{'─' * 80}")
        preview = msg.content[:60] + "..." if len(msg.content) > 60 else msg.content
        print(f"[{i}] 👤 USER: {preview}")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            tools_called = ", ".join([tc['name'] for tc in msg.tool_calls])
            print(f"[{i}] 🤖 AGENT: Calling {tools_called}")
        else:
            preview = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
            print(f"[{i}] 🤖 AGENT: {preview}")
            
    elif isinstance(msg, ToolMessage):
        first_line = msg.content.split('\n')[0]
        print(f"[{i}] 🔧 TOOL: {first_line}")

print(f"\n{'=' * 80}")
print(f"Total messages in state: {len(conversation['messages'])}")

# Count message types
human_count = sum(1 for m in conversation["messages"] if isinstance(m, HumanMessage))
ai_count = sum(1 for m in conversation["messages"] if isinstance(m, AIMessage))
tool_count = sum(1 for m in conversation["messages"] if isinstance(m, ToolMessage))

print(f"\nMessage breakdown:")
print(f"  👤 HumanMessages:  {human_count}")
print(f"  🤖 AIMessages:     {ai_count}")
print(f"  🔧 ToolMessages:   {tool_count}")

## 8. Visualize State Growth Over Time

Track how the conversation state grows with each turn.

In [ ]:
# Show state growth
print("STATE GROWTH ANALYSIS")
print("=" * 80)
print("\nTurn | Messages Added | Total Messages | Context Window")
print("-" * 80)

# Simulate tracking (approximation based on typical patterns)
turn_data = [
    ("Start", 0, 0, "Empty"),
    ("Turn 1", 4, 4, "Initial vehicle search"),
    ("Turn 2", 2, 6, "+ EMI calculation request"),
    ("Turn 3", 4, 10, "+ Vehicle comparison"),
    ("Turn 4", 5, 15, "+ Payment difference calculation")
]

for turn, added, total, context in turn_data:
    print(f"{turn:6s} | {added:14d} | {total:14d} | {context}")

print("\n" + "=" * 80)
print("Key Insight: Each turn adds 2-5 messages (query + response + tool calls/results)")
print("The LLM receives ALL previous messages on each turn, enabling context awareness.")

## 9. Test Context Understanding

Let's verify the agent truly understands context with a challenging follow-up.

In [ ]:
# Test context recall
print("CONTEXT RECALL TEST")
print("=" * 80)

user_query_5 = "Remind me, what was the monthly payment difference between the two SUVs we discussed?"
print(f"👤 User: {user_query_5}")
print(f"\n   💡 Testing if agent remembers: RAV4 vs Highlander comparison from Turn 4\n")

conversation["messages"].append(HumanMessage(content=user_query_5))
conversation = app.invoke(conversation)

print(f"🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 Final state size: {len(conversation['messages'])} messages")
print("\n✅ If the agent correctly recalls the payment difference, context is working!")

## 10. Inspect Message Details by Turn

Examine specific messages to see how context is maintained.

In [ ]:
# Show first few messages
print("FIRST 5 MESSAGES (Turn 1)")
print("=" * 80)
for i in range(min(5, len(conversation['messages']))):
    msg = conversation['messages'][i]
    msg_type = type(msg).__name__
    print(f"\n[{i}] {msg_type}:")
    if isinstance(msg, (HumanMessage, AIMessage)):
        preview = msg.content[:100] if msg.content else "[tool calls]"
        print(f"  {preview}...")
    elif isinstance(msg, ToolMessage):
        print(f"  {msg.content[:100]}...")

In [ ]:
# Show all messages
conversation["messages"]

## 11. Understanding Context Windows

### What Gets Passed to the LLM?

On each turn, the **entire** conversation history is sent to the LLM:

```python
# Turn 1
LLM receives: [HumanMessage₁]

# Turn 2  
LLM receives: [HumanMessage₁, AIMessage₁, ToolMessage₁, AIMessage₁_final, HumanMessage₂]

# Turn 3
LLM receives: [all messages from Turns 1-2, HumanMessage₃]
```

This is why the agent can understand:
- "that RAV4" (mentioned in Turn 1)
- "those two" (RAV4 and Highlander compared in Turn 3)
- "the two SUVs we discussed" (Turn 5 recalls Turn 4)

### Practical Considerations

**Advantages:**
- Full context awareness
- Natural conversation flow
- No need to repeat information

**Challenges:**
- Context window limits (models have max token limits)
- Increased latency (more tokens to process)
- Cost considerations (more tokens = higher API costs)

**Best Practices:**
- Summarize old messages after many turns
- Implement context pruning strategies
- Monitor state size in production applications

## 12. Pattern Comparison

Compare conversational context with other execution patterns.

In [ ]:
# Compare execution patterns
print("EXECUTION PATTERN COMPARISON")
print("=" * 80)

print("\n1. PARALLEL EXECUTION:")
print("   Pattern: Single AIMessage with multiple tool_calls")
print("   Message count: ~5")
print("   Use case: Independent tasks (search + calculate)")
print("   Turns: 1")

print("\n2. SEQUENTIAL EXECUTION:")
print("   Pattern: Multiple separate AIMessages with tool_calls")
print("   Message count: ~6")
print("   Use case: Dependent tasks (search result → EMI input)")
print("   Turns: 1")

print("\n3. CONVERSATIONAL CONTEXT:")
print("   Pattern: State accumulates across multiple user turns")
print(f"   Message count: {len(conversation['messages'])} (in our example)")
print("   Use case: Multi-turn car buying journey")
print(f"   Turns: {human_count}")

print("\n" + "=" * 80)
print("Key Difference: Conversational context spans MULTIPLE user queries,")
print("while parallel/sequential patterns complete within a SINGLE query.")

## 13. Additional Conversation Example

Start a fresh conversation to demonstrate another use case.

In [ ]:
# Start a new conversation
conversation2 = {"messages": []}

print("NEW CONVERSATION: Budget-Focused Buyer")
print("=" * 80)

# Turn 1
query1 = "I have a budget of $400/month. What Toyota vehicles can I afford?"
print(f"\nTurn 1: {query1}")
conversation2["messages"].append(HumanMessage(content=query1))
conversation2 = app.invoke(conversation2)
print(f"Response: {conversation2['messages'][-1].content[:150]}...")
print(f"State: {len(conversation2['messages'])} messages")

# Turn 2
query2 = "Which of those has the best fuel economy?"
print(f"\nTurn 2: {query2}")
print("   💡 'those' refers to vehicles mentioned in Turn 1")
conversation2["messages"].append(HumanMessage(content=query2))
conversation2 = app.invoke(conversation2)
print(f"Response: {conversation2['messages'][-1].content[:150]}...")
print(f"State: {len(conversation2['messages'])} messages")

# Turn 3
query3 = "Tell me more about its safety features."
print(f"\nTurn 3: {query3}")
print("   💡 'its' refers to the vehicle selected in Turn 2")
conversation2["messages"].append(HumanMessage(content=query3))
conversation2 = app.invoke(conversation2)
print(f"Response: {conversation2['messages'][-1].content[:150]}...")
print(f"State: {len(conversation2['messages'])} messages")

## Conclusion

In this notebook, you learned:

✅ **Conversational state management** - MessagesState accumulates ALL messages (HumanMessage, AIMessage, ToolMessage) across turns, enabling the agent to maintain full conversation context

✅ **Pronoun and reference resolution** - The agent successfully resolves references like "that RAV4" (Turn 2 → Turn 1), "those two" (Turn 4 → Turn 3), and "the two SUVs we discussed" (Turn 5 → Turn 4) by accessing conversation history

✅ **Multi-turn workflow patterns** - Built natural conversation flows for complex scenarios (4-5 turns, 15+ messages) where each turn builds on previous interactions, demonstrating real-world car buying journeys

✅ **State growth tracking** - Monitored how state expands from 0 → 4 → 6 → 10 → 15+ messages across turns, understanding that each turn adds 2-5 messages depending on tool usage

### Key Insights

**Context Accumulation Pattern:**
```
Turn 1: [H₁, AI₁, T₁, AI₁_final] = 4 messages
Turn 2: [Previous + H₂, AI₂_final] = 6 messages  
Turn 3: [Previous + H₃, AI₃, T₃, AI₃_final] = 10 messages
Turn 4: [Previous + H₄, AI₄, T₄, T₄, AI₄_final] = 15 messages
```

**Why This Matters:**
- **Natural UX**: Users don't need to repeat context ("the RAV4 you mentioned" vs just "it")
- **Intelligent Assistance**: Agent understands implicit references and maintains conversation thread
- **Real Conversations**: Enables multi-step processes like car buying, travel planning, or complex research

**Practical Considerations:**
- **Token Limits**: Most LLMs have context window limits (e.g., 32K, 128K tokens)
- **Cost**: More messages = more tokens processed = higher API costs
- **Latency**: Larger context windows take longer to process
- **Production Strategies**: Implement summarization, pruning, or context compression for long conversations

### Real-World Applications

This conversational pattern is essential for:
- **Customer Support**: Multi-turn troubleshooting with context
- **E-commerce**: Product search → comparison → purchase decision flow
- **Healthcare**: Symptom discussion → diagnosis → treatment planning
- **Education**: Tutoring sessions with progressive concept building
- **Financial Planning**: Budget analysis → investment options → recommendations

### Series Complete!

Congratulations! You've mastered the three core execution patterns in LangGraph:

1. **Parallel Execution** - Independent tasks executed simultaneously (5 messages)
2. **Sequential Execution** - Dependent tasks executed in order (6+ messages)  
3. **Conversational Context** - Multi-turn conversations with state accumulation (15+ messages)

Combined with vector similarity search and computational tools, you now have the complete toolkit to build sophisticated, production-ready agentic workflows with LangGraph!